<h1><center>Jupyter notebook for Seaside</center></h1>
Dylan R. Sanderson<br>
Last updated: April 16th, 2021

This notebook contains code to run infrastructure damage for Seaside. <br>
Four infrastructures are considered:
+ [Buildings](#bldg)
+ [Electric power network](#epn)
+ [Transportation (roads and bridges)](#trnsp)
+ [Water (water pipes and water facilities)](#wter)

### Importing pyIncore and other modules

In [ ]:
from pyincore import IncoreClient, Dataset, FragilityService, MappingSet
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage import CumulativeBuildingDamage
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.bridgedamage import BridgeDamage
from pyincore.analyses.roaddamage import RoadDamage
from pyincore.analyses.pipelinedamage import PipelineDamage
from pyincore.analyses.pipelinedamagerepairrate import PipelineDamageRepairRate

from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage
import os
import pandas as pd
client = IncoreClient()


### Specifying the earthquake and tsunami return periods
Seven recurrence intervals have been considered at Seaside
+ 100-yr
+ 250-yr
+ 500-yr
+ 1,000-yr
+ 2,500-yr
+ 5,000-yr
+ 10,000-yr

In [ ]:
rt = [100, 250, 500, 1000, 2500, 5000, 10000]
rt = [1000]

earthquake_hazard_dict = {100: "5dfa4058b9219c934b64d495", 
                  250: "5dfa41aab9219c934b64d4b2",
                  500: "5dfa4300b9219c934b64d4d0",
                  1000: "5dfa3e36b9219c934b64c231",
                  2500: "5dfa4417b9219c934b64d4d3", 
                  5000: "5dfbca0cb9219c101fd8a58d",
                 10000: "5dfa51bfb9219c934b68e6c2"}

tsunami_hazard_dict = {100: "5bc9e25ef7b08533c7e610dc", 
                  250: "5df910abb9219cd00cf5f0a5",
                  500: "5df90e07b9219cd00ce971e7",
                  1000: "5df90137b9219cd00cb774ec",
                  2500: "5df90761b9219cd00ccff258",
                  5000: "5df90871b9219cd00ccff273",
                  10000: "5d27b986b9219c3c55ad37d0"}


<a id='bldg'></a>
***
<h2><center>Building Infrastructure</center></h2>

#### Earthquake Building Damage
Running earthquake building damage.

In [ ]:
# initializing building damage and fragility service
bldg_dmg = BuildingDamage(client)   
fragility_service = FragilityService(client)

# defining building dataset (GIS point layer)
bldg_dataset_id = "5df40388b9219c06cf8b0c80"
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

# specifiying mapping id from fragilites to building types
mapping_id = "5d2789dbb9219c3c553c7977" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

bldg_dmg.set_parameter("hazard_type", "earthquake")
bldg_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\tearthquake: rt_{}' .format(rt_val))
    result_name = os.path.join('buildings_eq_{}yr' .format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]
    bldg_dmg.set_parameter("hazard_id", hazard_id)
    bldg_dmg.set_parameter("result_name", result_name)

    bldg_dmg.run_analysis()


#### Tsunami Building Damage
Running tsunami building damage

In [ ]:
# initializing pyincore building damage and fragility service 
bldg_dmg = BuildingDamage(client)
fragility_service = FragilityService(client)

# defining building dataset (GIS point layer)
bldg_dataset_id = "5df40388b9219c06cf8b0c80"
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)

# specifiying mapping id from fragilites to building types
mapping_id = "5d279bb9b9219c3c553c7fba" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

bldg_dmg.set_parameter("hazard_type", "tsunami")
bldg_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('buildings_tsu_{}yr' .format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]
    bldg_dmg.set_parameter("hazard_id", hazard_id)
    bldg_dmg.set_parameter("result_name", result_name)

    bldg_dmg.run_analysis()

#### Cumulative Building Damage
This cell must be ran after the above. It relies on output from the earthquake and tsunami building damage modules. 

In [ ]:
# initializing pyIncore cumulative building damage
cumulative_bldg_dmg = CumulativeBuildingDamage(client)
cumulative_bldg_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\tcumulative: rt_{}' .format(rt_val))
    # reading in damage results from above analysis
    eq_damage_results_csv = os.path.join('buildings_eq_{}yr.csv' .format(rt_val))
    tsu_damage_results_csv = os.path.join('buildings_tsu_{}yr.csv' .format(rt_val))

    # loading datasets from CSV files into pyincore
    eq_damage_dataset = Dataset.from_file(eq_damage_results_csv, "ergo:buildingDamageVer5")
    tsu_damage_dataset = Dataset.from_file(tsu_damage_results_csv, "ergo:buildingDamageVer5")

    cumulative_bldg_dmg.set_input_dataset("eq_bldg_dmg", eq_damage_dataset)
    cumulative_bldg_dmg.set_input_dataset("tsunami_bldg_dmg", tsu_damage_dataset)

    # defining path to output 
    result_name = os.path.join('buildings_cumulative_{}yr' .format(rt_val))
    cumulative_bldg_dmg.set_parameter("result_name", result_name)

    # running analysis
    cumulative_bldg_dmg.run_analysis()

<a id='epn'></a>
***
<h2><center>Electric Power Network</center></h2>
The electric power network consists of electric poles and substation

#### Earthquake Electric Damage

In [ ]:
# initializing epf damage and fragility service
epf_dmg = EpfDamage(client)
fragility_service = FragilityService(client)

# defining electric pole dataset (GIS point layer)
poles_ss_id = "5d263f08b9219cf93c056c68"     # elelctric power poles and substation
epf_dmg.load_remote_input_dataset("epfs", poles_ss_id)

# Fragility Mapping on incore-service
# mapping_id = "5d489aa1b9219c0689f1988e" # 5 DS
mapping_id = "6079f6a7ef881f48a4ace306" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
epf_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

epf_dmg.set_parameter("hazard_type", 'earthquake')
epf_dmg.set_parameter("num_cpu", 1)
epf_dmg.set_parameter('fragility_key', "pga")

for rt_val in rt:
    print('\teartquake: rt_{}' .format(rt_val))
    result_name = os.path.join('electric_eq_{}yr_dmg'.format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]

    epf_dmg.set_parameter("hazard_id", hazard_id)
    epf_dmg.set_parameter("result_name", result_name)

    # Run Analysis
    epf_dmg.run_analysis()

#### Tsunami Electric Damage

In [ ]:
# Initializing EPF damage and fragility service
epf_dmg = EpfDamage(client)
fragility_service = FragilityService(client)

# defining electric poles and substation dataset (GIS point layer)
poles_ss_id = "5d263f08b9219cf93c056c68"
epf_dmg.load_remote_input_dataset("epfs", poles_ss_id)

# Fragility Mapping on incore-service
# mapping_id = "5d31eb7fb9219c6d66398445" # 5 DS
mapping_id = "6079c5e06799d908861f177c" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
epf_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

epf_dmg.set_parameter("hazard_type", "tsunami")
epf_dmg.set_parameter("num_cpu", 1)
epf_dmg.set_parameter('fragility_key', "Non-Retrofit inundationDepth Fragility ID Code")

for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('electric_tsu_{}yr_dmg'.format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]

    epf_dmg.set_parameter("hazard_id", hazard_id)
    epf_dmg.set_parameter("result_name", result_name)

    # Run Analysis
    epf_dmg.run_analysis()

#### Cumulative Electric Damage
Does not exist in pyIncore

<a id='trnsp'></a>
***
<h2><center>Transportation Network</center></h2>

#### Earthquake Road Damage

In [ ]:
# Initializing road damage and fragility service
road_dmg = RoadDamage(client)
fragility_service = FragilityService(client)

# loading in road dataset
road_dataset_id = "5d25118eb9219c0692cd7527"
road_dmg.load_remote_input_dataset("roads", road_dataset_id)

# seaside road fragility mappng for EQ
# mapping_id = "5d545b0bb9219c0689f1f3f4" # 5 DS
mapping_id = "6079ec1aef881f48a4ac586c" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
road_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

road_dmg.set_parameter("hazard_type", 'earthquake')
road_dmg.set_parameter("num_cpu", 1)
road_dmg.set_parameter("fragility_key", 'pgd')

for rt_val in rt:
    print('\tearthquake rt_{}' .format(rt_val))
    result_name = os.path.join('road_eq_{}yr_dmg' .format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]

    road_dmg.set_parameter("hazard_id", hazard_id)
    road_dmg.set_parameter("result_name", result_name)

    road_dmg.run_analysis()


##### Earthquake Bridge Damage

In [ ]:
# initializing bridge damage and fragility service
brdg_dmg = BridgeDamage(client)
fragility_service = FragilityService(client)

# Seaside bridges
bridge_dataset_id = "5d6ede5db9219c34b56fc20b"
brdg_dmg.load_remote_input_dataset("bridges", bridge_dataset_id)

# Set analysis parameters

# seaside road fragility mappng for EQ
# mapping_id = "5d55c3a1b9219c0689f1f898" # 5 DS
mapping_id = "6079f5435b609c56229bf85c" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
brdg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

brdg_dmg.set_parameter("hazard_type", 'earthquake')
brdg_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\tearthquake: rt_{}' .format(rt_val))
    result_name = os.path.join('bridge_eq_{}yr_dmg' .format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]
    brdg_dmg.set_parameter("hazard_id", hazard_id)
    brdg_dmg.set_parameter("result_name", result_name)

    brdg_dmg.run_analysis()

#### Tsunami Road Damage

In [ ]:
# Initializing road damage and fragility service
road_dmg = RoadDamage(client)
fragility_service = FragilityService(client)

# loading road dataset
road_dataset_id = "5d25118eb9219c0692cd7527"
road_dmg.load_remote_input_dataset("roads", road_dataset_id)

# seaside road fragility mappng for tsunami
mapping_id = "5d274fd8b9219c3c553c71ff" # 5 DS
# mapping_id = "6079c3bc6799d908861f16d4" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
road_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

road_dmg.set_parameter("hazard_type", 'tsunami')
road_dmg.set_parameter("num_cpu", 1)
road_dmg.set_parameter("fragility_key", "Non-Retrofit inundationDepth Fragility ID Code")


for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('road_tsu_{}yr_dmg' .format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]

    road_dmg.set_parameter("hazard_id", hazard_id)
    road_dmg.set_parameter("result_name", result_name)

    road_dmg.run_analysis()

##### Tsunami Bridge Damage

In [ ]:
# Initialzing bridge damage and fragility service
brdg_dmg = BridgeDamage(client)
fragility_service = FragilityService(client)

# Seaside bridges
bridge_dataset_id = "5d6ede5db9219c34b56fc20b"
brdg_dmg.load_remote_input_dataset("bridges", bridge_dataset_id)

# seaside road fragility mappng for EQ
mapping_id = "5d275000b9219c3c553c7202"	# 5 DS
# mapping_id = "6079c41a6799d908861f16fe" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
brdg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

brdg_dmg.set_parameter("hazard_type", 'tsunami')
brdg_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('bridge_tsu_{}yr_dmg' .format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]
    brdg_dmg.set_parameter("hazard_id", hazard_id)
    brdg_dmg.set_parameter("result_name", result_name)

    brdg_dmg.run_analysis()

#### Cumulative Transportation Damage
Does not exist in pyIncore

<a id='wter'></a>
***
<h2><center>Water Network</center></h2>


#### Earthquake Pipe Damage

In [ ]:
# Initializing pipeline damage and fragility service
pipeline_dmg = PipelineDamageRepairRate(client)
fragility_service = FragilityService(client)

# Seaside pipes
pipe_dataset_id = "5d2666b5b9219c3c5595ee65"
pipeline_dmg.load_remote_input_dataset("pipeline", pipe_dataset_id)

# seaside pipe fragility mappng for EQ
mapping_id = "5b47c227337d4a38464efea8"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
pipeline_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

pipeline_dmg.set_parameter("hazard_type", 'earthquake')
pipeline_dmg.set_parameter("fragility_key",'pgv')
# pipeline_dmg.set_parameter("use_liquefaction",True)
pipeline_dmg.set_parameter("num_cpu", 1)


for rt_val in rt:
    print('\tearthquake: rt_{}' .format(rt_val))
    result_name = os.path.join('pipe_eq_{}yr_dmg' .format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]

    pipeline_dmg.set_parameter("hazard_id", hazard_id)
    pipeline_dmg.set_parameter("result_name",result_name)

    # Run pipeline damage analysis
    result = pipeline_dmg.run_analysis()



#### Tsunami Pipe Damage

In [ ]:
# Initialzing pipeline damage and fragility service
pipeline_dmg = PipelineDamage(client)
fragility_service = FragilityService(client)

# Seaside pipes
pipe_dataset_id = "5d2666b5b9219c3c5595ee65"
pipeline_dmg.load_remote_input_dataset("pipeline", pipe_dataset_id)

# seaside pipe fragility mappng for tsunami
mapping_id = "5d320a87b9219c6d66398b45" # 5 DS
# mapping_id = "6079b7156799d908861e413b" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
pipeline_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

pipeline_dmg.set_parameter("hazard_type", 'tsunami')
pipeline_dmg.set_parameter("fragility_key", "Non-Retrofit inundationDepth Fragility ID Code")
pipeline_dmg.set_parameter("num_cpu", 1)

for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('pipe_tsu_{}yr_dmg' .format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]

    pipeline_dmg.set_parameter("hazard_id", hazard_id)
    pipeline_dmg.set_parameter("result_name",result_name)

    # Run pipeline damage analysis
    result = pipeline_dmg.run_analysis()



#### Water Facility Earthquake Damage

In [ ]:
# Initializing water facility damage and fragility service
wterfclty_dmg = WaterFacilityDamage(client)
fragility_service = FragilityService(client)

# loading input dataset
fclty_dataset_id = "5d266507b9219c3c5595270c"
wterfclty_dmg.load_remote_input_dataset("water_facilities", fclty_dataset_id)

# wterfclty Fragility Mapping on incore-service
mapping_id = "5d39e010b9219cc18bd0b0b6" # 5 DS
# mapping_id = "6079f7825b609c56229bf976" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
wterfclty_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

wterfclty_dmg.set_parameter("hazard_type", 'earthquake')
wterfclty_dmg.set_parameter("fragility_key",'pga')
wterfclty_dmg.set_parameter("num_cpu", 1)


for rt_val in rt:
    print('\tearthquake: rt_{}' .format(rt_val))
    result_name = os.path.join('wterfclty_eq_{}yr_dmg' .format(rt_val))
    hazard_id = earthquake_hazard_dict[rt_val]

    wterfclty_dmg.set_parameter("hazard_id", hazard_id)
    wterfclty_dmg.set_parameter("result_name",result_name)

    # Run pipeline damage analysis
    result = wterfclty_dmg.run_analysis()


#### Water Facility Tsunami Damage

In [ ]:
# initialzing water facility damage and fragility service
wterfclty_dmg = WaterFacilityDamage(client)
fragility_service = FragilityService(client)

# loading input dataset
fclty_dataset_id = "5d266507b9219c3c5595270c"
wterfclty_dmg.load_remote_input_dataset("water_facilities", fclty_dataset_id)

# wterfclty Fragility Mapping on incore-service
mapping_id = "5d31f737b9219c6d66398521" # 5 DS
# mapping_id = "6079b8a66799d908861e4bf0" # 4 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
wterfclty_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

wterfclty_dmg.set_parameter("hazard_type", 'tsunami')
wterfclty_dmg.set_parameter("fragility_key","Non-Retrofit inundationDepth Fragility ID Code")
wterfclty_dmg.set_parameter("num_cpu", 1)


for rt_val in rt:
    print('\ttsunami: rt_{}' .format(rt_val))
    result_name = os.path.join('wterfclty_tsu_{}yr_dmg' .format(rt_val))
    hazard_id = tsunami_hazard_dict[rt_val]

    wterfclty_dmg.set_parameter("hazard_id", hazard_id)
    wterfclty_dmg.set_parameter("result_name",result_name)

    # Run pipeline damage analysis
    result = wterfclty_dmg.run_analysis()


#### Cumulative Water Damage
Does not exist in pyIncore